In [1]:
import numpy as np
import requests

In [2]:
## en_US names
response = requests.get('https://raw.githubusercontent.com/Kengxxiao/ArknightsGameData/master/en_US/gamedata/excel/item_table.json')
itemdict = response.json()

ITEM_NAME_DICT = {} # itemId -> itemName

for itemId, info in itemdict['items'].items():
    itemName = info['name']
    #ITEM_NAME_DICT[itemName] = itemId
    ITEM_NAME_DICT[itemId] = itemName

In [3]:
## Caveman brain function, but it works
def sum_skill_slice(array):
    total_ids = []
    total_counts = []

    for ids, counts in array:
        for i in range(len(ids)):
            total_ids.append(ids[i])
            total_counts.append(counts[i])

    unique_ids = np.unique(total_ids)
    unique_counts = np.zeros(len(unique_ids))

    for i in range(len(total_ids)):
        idx = np.where(unique_ids == total_ids[i])[0]
        unique_counts[idx] += total_counts[i]
    
    unique_ids = np.trim_zeros(unique_ids)
    unique_counts = np.trim_zeros(unique_counts)
    return np.array([unique_ids, unique_counts], dtype="uint32").transpose()


## Source files ver

In [6]:
response = requests.get("https://raw.githubusercontent.com/Kengxxiao/ArknightsGameData/master/zh_CN/gamedata/excel/character_table.json")
chartab = response.json()

response = requests.get("https://raw.githubusercontent.com/Kengxxiao/ArknightsGameData/master/en_US/gamedata/excel/character_table.json")
chartab_en = response.json()

response = requests.get('https://raw.githubusercontent.com/Kengxxiao/ArknightsGameData/master/zh_CN/gamedata/excel/building_data.json')
craftdict = response.json()

## en_US names
response = requests.get('https://raw.githubusercontent.com/Kengxxiao/ArknightsGameData/master/en_US/gamedata/excel/item_table.json')
itemdict = response.json()

ITEM_NAME_DICT = {} # itemName -> itemId
REVERSE_ITEM_NAME_DICT = {} # itemId -> itemName

for itemId, info in itemdict['items'].items():
    itemName = info['name']
    ITEM_NAME_DICT[itemName] = itemId
    REVERSE_ITEM_NAME_DICT[itemId] = itemName

n_items = len(ITEM_NAME_DICT.keys())

In [ ]:
op_name_dict = {}
op_name_dict_en = {}
op_name_dict_en_rev = {}

n_operators = len(chartab)

op_ids = np.empty(n_operators, dtype="U32")

elite_costs = np.zeros((n_operators,2), dtype=[
    ("item_id", "uint32", 3),
    ("count", "uint32", 3),
])

skill_costs = np.zeros((n_operators, 6), dtype=[
    ("item_id", "uint32", 3),
    ("count", "uint32", 3),
])

mastery_costs = np.zeros((n_operators, 3, 3), dtype=[
    ("item_id", "uint32", 3),
    ("count", "uint32", 3),
])

#print(elite_costs)

for char_id, v in chartab_en.items():
    op_name_dict_en[char_id] = v["name"]
    op_name_dict_en_rev[v["name"]] = char_id

for char_idx, (char_id, v) in enumerate(chartab.items()):
    
    ## Name (CN)
    op_name_dict[char_id] = v["name"]
    op_ids[char_idx] = char_id 
    
    ## Promotion costs
    for p_idx, p in enumerate(v["phases"]):
        cost_arr = p["evolveCost"]
        if p["evolveCost"] is not None:
            for c_idx, c in enumerate(cost_arr):
                elite_costs[char_idx][p_idx-1]["item_id"][c_idx] = c["id"]
                elite_costs[char_idx][p_idx-1]["count"][c_idx] = c["count"]
            
            
    ## General skills
    for s_idx, s in enumerate(v["allSkillLvlup"]):
        if s["lvlUpCost"] is not None:
            for c_idx, c in enumerate(s["lvlUpCost"]):
                skill_costs[char_idx][s_idx]["item_id"][c_idx] = c["id"]
                skill_costs[char_idx][s_idx]["count"][c_idx] = c["count"]
        
   ## Skill masteries
    for s_idx, s in enumerate(v["skills"]):
        for m_idx, m in enumerate(s["levelUpCostCond"]):
            if m["levelUpCost"] is not None:
                for c_idx, c in enumerate(m["levelUpCost"]):
                    mastery_costs[char_idx][s_idx][m_idx]["item_id"][c_idx] = c["id"]
                    mastery_costs[char_idx][s_idx][m_idx]["count"][c_idx] = c["count"]
         

In [8]:
def get_op_stuff(name: str):
    char_id = op_name_dict_en_rev[name]
    char_idx = np.where(op_ids == char_id)[0][0]
    
    e_cost = elite_costs[char_idx]
    s_cost = skill_costs[char_idx]
    m_cost = mastery_costs[char_idx]
    
    return e_cost, s_cost, m_cost

In [ ]:
e_cost, s_cost, m_cost = get_op_stuff("Angelina")

#print(e_cost)
#print(s_cost)
print(m_cost)

In [ ]:
ret = sum_skill_slice(m_cost[0])
print(ret)
print(ITEM_NAME_DICT[str(ret[6][0])])

In [ ]:
op_name = "Nearl"
elite_range = (0, 2)
skill_range = (1, 7)
mastery_range = [
    (0, 0),
    (0, 0),
    (0, 3)
]

mats_combined = np.zeros(2+6+9, dtype=[
    ("item_id", "uint32", 3),
    ("count", "uint32", 3),
])


e_cost, s_cost, m_cost = get_op_stuff(op_name)

j = 0
for v in e_cost[elite_range[0] : elite_range[1]]:
    mats_combined[j] = v
    j += 1
    
for v in s_cost[skill_range[0]-1 : skill_range[1]-1]:
    mats_combined[j] = v
    j += 1

for i in range(len(mastery_range)):
    for v in m_cost[i][mastery_range[i][0] : mastery_range[i][1]]:
        mats_combined[j] = v
        j += 1

mats_combined = sum_skill_slice(mats_combined)
print(mats_combined)

In [ ]:
for i, c in mats_combined:
    print("{}: {}".format(REVERSE_ITEM_NAME_DICT[str(i)], c))

In [ ]:
print("MATERIAL_IDS = [")
for k in ITEM_NAME_DICT.keys():
    print("\t\"{}\",".format(k))
print("]")

In [ ]:
print(ITEM_NAME_DICT["LMD"])